In [1]:
import pandas as pd
import os
import sqlite3

In [2]:
os.listdir()

['order_reviews.csv',
 'products.csv',
 'orders.csv',
 'product_category_name_translation.csv',
 'Untitled.ipynb',
 'customer.csv',
 'order_payments.csv',
 '.ipynb_checkpoints',
 'order_items.csv',
 'seller.csv']

In [3]:
df = pd.read_csv('seller.csv')
df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [4]:
#database
db = '/Users/bimie/Desktop/repos/Dynamic-Analytic-Dashboard-With-Plotly/ecommerce'
sqliteConnection = sqlite3.connect(db)

In [5]:
cursor = sqliteConnection.cursor()

In [6]:
query = '''SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ="table" AND 
    name NOT LIKE "sqlite_%";'''
cursor.execute(query)
result = cursor.fetchall()
df = pd.read_sql(query, sqliteConnection )
df.head(10)

,name
0,customer
1,seller
2,products
3,orders
4,order_items
5,order_payments
6,product_category_name_translation


In [7]:
def run_query(query):
    DATA = []
    cursor.execute(query)
    result = cursor.fetchall()
    for d in pd.read_sql(query, sqliteConnection, chunksize=100):
        DATA.append(d)
    df = pd.concat(DATA)
    return df

In [8]:
query = ''' SELECT pt.product_category_name_english, COUNT(oi.product_id) AS 'number_ordered'  FROM order_items oi
        LEFT JOIN products p on p.id = oi.product_id
        LEFT JOIN product_category_name_translation pt ON p.category_name = pt.product_category_name
         GROUP BY p.category_name
          ORDER BY 2 DESC 
          LIMIT 10'''
top_products = run_query(query)

In [10]:
top_products

,product_category_name_english,number_ordered
0,bed_bath_table,11115
1,health_beauty,9670
2,sports_leisure,8641
3,furniture_decor,8334
4,computers_accessories,7827
5,housewares,6964
6,watches_gifts,5991
7,telephony,4545
8,garden_tools,4347
9,auto,4235


In [46]:
query = ''' SELECT pt.product_category_name_english AS product_name, strftime('%Y', order_purchase_timestamp),
                            COUNT(oi.product_id) AS 'number_ordered'  FROM order_items oi
                    LEFT JOIN products p on p.id = oi.product_id
                    LEFT JOIN product_category_name_translation pt ON p.category_name = pt.product_category_name
                    LEFT JOIN orders o ON o.id = oi.order_id
                    WHERE strftime('%Y', order_purchase_timestamp) =  '2018'
                    GROUP BY p.category_name
                    ORDER BY 2 DESC 
                    LIMIT 10 '''
df = run_query(query)
df.head()

,product_name,"strftime('%Y', order_purchase_timestamp)",number_ordered
0,housewares,2018,4046
1,fixed_telephony,2018,102
2,telephony,2018,2336
3,tablets_printing_image,2018,22
4,signaling_and_security,2018,153


In [41]:
df.isnull().sum()

id                               0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
year                             0
dtype: int64

In [22]:
query = ''' SELECT SUM(payment_value) FROM order_payments op 
            LEFT JOIN orders o ON o.id = op.order_id
        '''
df = run_query(query)
df

,SUM(payment_value)
0,1.600887e+07


In [14]:
df['order_id'].value_counts()

fa65dad1b0e818e3ccc5cb0e39231352    29
ccf804e764ed5650cd8759557269dc13    26
285c2e15bebd4ac83635ccc563dc71f4    22
895ab968e7bb0d5659d16cd74cd1650c    21
fedcd9f7ccdc8cba3a18defedd1a5547    19
                                    ..
6d2a30c9b7dcee3ed507dc9a601f99e7     1
a7737f6d9208dd56ea498a322ed3c37f     1
646e62df54f3e236eb6d5ff3b31429b8     1
e115da7a49ec2acf622e1f31da65cfb9     1
28bbae6599b09d39ca406b747b6632b1     1
Name: order_id, Length: 99440, dtype: int64

In [5]:
import plotly
import plotly.graph_objs as go

In [4]:
pip install plotly


     |████████████████████████████████| 15.2 MB 697 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
def plot1():
    data = [go.Bar(x=top_products['product_category_name_english'], y=top_products['number_ordered'])]
    layout = go.Layout(title='Plot 1')
    return data, go.Figure(data=data, layout=layout)

def plot2():
    data = [go.Scatter(x=[1, 2, 3], y=[4, 5, 6])]
    layout = go.Layout(title='Plot 2')
    return go.Figure(data=data, layout=layout)


def index():
    plot1_json = plot1().to_json()
    plot2_json = plot2().to_json()
    return render_template('index.html', plot1_json=plot1_json, plot2_json=plot2_json)

In [21]:
top_products.columns

Index(['product_category_name_english', 'number_ordered'], dtype='object')

In [23]:
a, b = plot1()

In [24]:
a

[Bar({
     'x': array(['bed_bath_table', 'health_beauty', 'sports_leisure', 'furniture_decor',
                 'computers_accessories', 'housewares', 'watches_gifts', 'telephony',
                 'garden_tools', 'auto'], dtype=object),
     'y': array([11115,  9670,  8641,  8334,  7827,  6964,  5991,  4545,  4347,  4235])
 })]

In [25]:
b